In [1]:
# importar hdfs
from hdfs import InsecureClient

from pyspark.sql import SparkSession
from operator import add


#abro una sesion con un nombre
#hay que tener cuidado que los workers de spark tengan la cantidad de Mb para ejecutar
spark = SparkSession\
.builder\
.config("spark.submit.deployMode", "client")\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "1g")\
.config("spark.driver.memory", "1g")\
.config("spark.executor.memoryOverhead", "1g")\
.appName("probandospark6rdd")\
.master("spark://bigdata-srv.fi.uncoma.edu.ar:7077")\
.getOrCreate()

spark.sparkContext._conf.getAll()  # ver la configuración


[('spark.app.name', 'probandospark6rdd'),
 ('spark.driver.host', '192.168.1.42'),
 ('spark.sql.warehouse.dir',
  'file:/home/usuarioFAI/ejemplosPython/spark-warehouse'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '1g'),
 ('spark.driver.port', '34087'),
 ('spark.app.startTime', '1620874913349'),
 ('spark.executor.memory', '1g'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.executor.instances', '1'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.master', 'spark://bigdata-srv.fi.uncoma.edu.ar:7077'),
 ('spark.app.id', 'app-20210513000155-0006'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.executor.memoryOverhead', '1g')]

In [2]:

     
def otraPruebaRDDdesdeHDFS():  
    distFile = spark.sparkContext.textFile('hdfs://127.0.0.1:9000/cursoFAI/nba.csv')
    counts=distFile.flatMap(lambda x: x.split(" ")).map(lambda x: (x,1)).reduceByKey(add)
    
    #si existe borrarla
    #conectarme
    client = InsecureClient('http://localhost:9870', user='usuarioFAI')
    #Borrar un archivo o carpeta - sino esta no hace nada
    borrar = client.delete('/cursoFAI/fuente1Modificada.txt',recursive=True)
        
    counts.saveAsTextFile('hdfs://127.0.0.1:9000/cursoFAI/fuente1Modificada.txt')
    #esto que comente lista el archivo y es muy grande!
    #for (word, count) in output:
     #   print(word,",",count)
    
otraPruebaRDDdesdeHDFS()

In [3]:
#cierro la sesión
spark.sparkContext.stop()